In [21]:
import os
import sys
from scipy.spatial.transform import Rotation as R

%reload_ext autoreload
%autoreload 2

current_directory = os.getcwd()
master_thesis_path = os.path.join(os.path.sep, *current_directory.split(os.path.sep)[:-2])

# Print the master thesis path
print(master_thesis_path)

sys.path.append(os.path.join(master_thesis_path, 'neuralblox'))
sys.path.append(os.path.join(master_thesis_path, 'neuralblox', 'configs'))

#cd to neuralblox folder
os.chdir(os.path.join(master_thesis_path, 'neuralblox'))

import torch
import numpy as np

import open3d as o3d
from src.utils.debug_utils import *
is_cuda = (torch.cuda.is_available())
device = torch.device("cuda" if is_cuda else "cpu")
print(device)
import src.neuralblox.helpers.visualization_utils as vis_utils
import src.neuralblox.helpers.sequential_trainer_utils as st_utils
import src.neuralblox.helpers.metrics_utils as metrics_utils

if 'robin' in os.getcwd():
    bool_location = 1
    cfg_path = 'configs/evaluation_cfg.yaml'
    print(f'On home')
elif 'cluster' in os.getcwd():
    bool_location = 2
    print(f'On euler')
else:
    bool_location = 0
    cfg_path = 'configs/evaluation_cfg_local.yaml'
    print(f'On local')

cfg_default_path = 'configs/default.yaml'

cfg = config.load_config(cfg_path, cfg_default_path)


/home/robin/Dev/MasterThesis/GithubRepos/master_thesis
cuda
On home


In [33]:
import numpy as np 
import torch 
import open3d as o3d
import trimesh
import copy
import os 

path_dir = '/media/robin/T7/neuralblox/evaluation/processed_data'
og = False
dispatch = True
n_points = 8192

path = os.path.join(path_dir, f'data_saving_{n_points}_1_og{og}_dispatch-{dispatch}_.pth')
data = torch.load(path)
import yaml

def load_config(path):
    with open(path, 'r') as stream:
        return yaml.safe_load(stream)
cfg = '/home/robin/Dev/MasterThesis/GithubRepos/master_thesis/neuralblox/configs/evaluation_cfg.yaml'
cfg = load_config(cfg)

terrains_path = cfg['data']['scene_folder']

print(data.keys())

dict_keys(['inputs', 'inputs.occ', 'query_points', 'logits_sampled', 'model_infos', 'transform', 'cfg'])


/tmp/ipykernel_640804/4071480250.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(path)


In [34]:
query_points = data['query_points']
logits_sampled = data['logits_sampled']

occ_sampled = 1 / (1 + np.exp(-logits_sampled))
occ_sampled = occ_sampled > 0.2

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(query_points[occ_sampled == 1])
pcd.paint_uniform_color(np.random.rand(3))

inputs = data['inputs']
inputs_occ = data['inputs.occ']

pcd_inputs = o3d.geometry.PointCloud()
pcd_inputs.points = o3d.utility.Vector3dVector(inputs[inputs_occ == 1])
pcd_inputs.paint_uniform_color(np.random.rand(3))

terrain = data['model_infos'][0]['category'][0]
transform = data['transform']

gt_mesh_o3d, gt_mesh_points = metrics_utils.load_ground_truth(cfg, terrain)
gt_mesh_o3d, gt_mesh_points = metrics_utils.apply_transformations(transform, gt_mesh_o3d, gt_mesh_points)
gt_mesh_o3d.compute_vertex_normals()
o3d.visualization.draw_geometries([pcd, gt_mesh_o3d, pcd_inputs])

In [37]:
mesh_path = os.path.join(path_dir, f'mesh_data_saving_8192_1_ogTrue_dispatch-True_.pth.obj')
mesh_og = o3d.io.read_triangle_mesh(mesh_path)
mesh_og.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_og, gt_mesh_o3d, pcd])

In [44]:
dataset = config.get_dataset(cfg['generation']['dataset'], cfg)

loader = torch.utils.data.DataLoader(
    dataset, batch_size=1, 
    num_workers=cfg['training']['n_workers'], 
    shuffle=True,
    collate_fn=data_src.collate_remove_none,
    worker_init_fn=data_src.worker_init_fn)

batch = next(iter(loader))
batch_subsampled_reduced = metrics_utils.process_batch(batch, cfg, n_max_inputs = 8192)
for key in batch_subsampled_reduced.keys():
    try:
        print(f'{key}: {batch_subsampled_reduced[key].shape}')
    except:
        print(f'{key}: {batch_subsampled_reduced[key]}')

group size: 1, total models: 5
added group, size: 1, total groups: 0
added group, size: 1, total groups: 1
added group, size: 1, total groups: 2
added group, size: 1, total groups: 3
added group, size: 1, total groups: 4
points: torch.Size([1, 20, 40960, 3])
points.occ: torch.Size([1, 20, 40960])
inputs: torch.Size([1, 20, 8192, 3])
inputs.occ: torch.Size([1, 20, 8192])
transform: [tensor([[ 0.0000, 40.6471,  0.0000]], dtype=torch.float64), tensor([[-0.0220,  0.0709,  0.1219]])]
model_infos: [{'category': ['2024-08-21_16-54-30'], 'model': ['000000']}]


In [51]:
print(f'Generating meshes')
cfg = config.load_config(cfg_path, cfg_default_path)
from src.neuralblox import config_generators

generator_robot = config_generators.get_generator_sequential(cfg, device=device)
print(data['inputs'].shape)
print(data['inputs.occ'].shape)
batch_subsampled_reduced = {
    'inputs': data['inputs'].to(torch.float32),
    'inputs.occ': data['inputs.occ'],
    'points': data['inputs'].to(torch.float32),
    'points.occ': data['inputs.occ']
}
idx_vis = 10 

with torch.no_grad():
    mesh_list, inputs_frame_list, times, logits = generator_robot.generate_mesh_at_index(batch_subsampled_reduced, idx_vis, 0, memory_keep=False, generate_logits=True)   


Generating meshes
/media/robin/T7/report/models/model_backbone.pt
=> Loading checkpoint from local file...


/home/robin/Dev/MasterThesis/GithubRepos/master_thesis/neuralblox/src/checkpoints.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(filename)


/media/robin/T7/report/models/model_merging.pt
=> Loading checkpoint from local file...
Location: home
torch.Size([1, 20, 8192, 3])
torch.Size([1, 20, 8192])


/home/robin/anaconda3/envs/neuralblox_3_8_v2/lib/python3.8/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3609.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Generating mesh at index 10
Decoding latent codes from 106 voxels


In [55]:
vis_utils.visualize_mesh_and_points(inputs_frame_list, mesh_list, 1.0, add_geos = [mesh_og])


TypeError: visualize_mesh_and_points() got an unexpected keyword argument 'add_geos'